In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

2023-07-20 21:40:01.723541: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-20 21:40:01.724947: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-20 21:40:01.756281: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-20 21:40:01.756887: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-20 21:40:02.313485: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
df = pd.read_excel('./data.xlsx')
df = df.rename(columns={'Conclusion': 'y'})
df = df.drop(columns=['Company', 'Symbol', 'Chart'])
df = df.replace('-', np.nan)
print(df.shape)
df.head()

(899, 427)


,TSE_FY,RE_FY,y,Custom-Return,PtoB,PtoS,PtoFCFF,PtoE_ttm,PtoFCFE,PtoNAV,...,OP_FY,TA_MRQ,TA_FY,TL_MRQ,TL_FY,TSE_MRQ,RE_MRQ,FE_TTM,FE_MRQ,FE_FY
0,4.371690e+11,-1.348210e+14,308.395650,-0.046138,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.781578e+14,5.709565e+14,6.205788e+14,5.705193e+14,-4.242109e+13,-1.731840e+14,7.986821e+12,4.155223e+12,1.144012e+12
1,-1.125286e+15,-1.142159e+15,1.014995,NaN,NaN,NaN,48.5539,NaN,27.0587,NaN,...,NaN,3.158876e+15,3.158876e+15,4.284162e+15,4.284162e+15,-1.125286e+15,-1.142159e+15,8.043798e+13,3.522180e+13,8.043798e+13
2,1.995942e+15,2.536933e+14,0.127105,0.604888,1.1937,9.4587,NaN,10.2428,NaN,0.8344,...,2.219208e+14,1.895281e+15,2.053317e+15,1.231507e+14,5.737468e+13,1.772130e+15,2.437543e+13,1.652797e+13,8.066030e+12,1.119782e+13
3,4.573182e+13,2.767290e+12,0.060511,0.638978,1.3568,265.4196,NaN,471.9549,NaN,1.3550,...,1.967870e+11,4.591844e+13,4.622727e+13,5.911390e+11,4.954510e+11,4.532730e+13,2.369800e+12,NaN,NaN,NaN
4,-4.739843e+13,-1.610339e+14,3.397452,0.306068,NaN,0.8690,NaN,NaN,NaN,NaN,...,-7.346602e+13,1.371765e+14,1.371765e+14,1.845749e+14,1.845749e+14,-4.739843e+13,-1.610339e+14,4.607902e+12,1.666914e+12,4.607902e+12


In [3]:
le = LabelEncoder()
df['Industry'] = le.fit_transform(df['Industry'])
df.head()

,TSE_FY,RE_FY,y,Custom-Return,PtoB,PtoS,PtoFCFF,PtoE_ttm,PtoFCFE,PtoNAV,...,OP_FY,TA_MRQ,TA_FY,TL_MRQ,TL_FY,TSE_MRQ,RE_MRQ,FE_TTM,FE_MRQ,FE_FY
0,4.371690e+11,-1.348210e+14,308.395650,-0.046138,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.781578e+14,5.709565e+14,6.205788e+14,5.705193e+14,-4.242109e+13,-1.731840e+14,7.986821e+12,4.155223e+12,1.144012e+12
1,-1.125286e+15,-1.142159e+15,1.014995,NaN,NaN,NaN,48.5539,NaN,27.0587,NaN,...,NaN,3.158876e+15,3.158876e+15,4.284162e+15,4.284162e+15,-1.125286e+15,-1.142159e+15,8.043798e+13,3.522180e+13,8.043798e+13
2,1.995942e+15,2.536933e+14,0.127105,0.604888,1.1937,9.4587,NaN,10.2428,NaN,0.8344,...,2.219208e+14,1.895281e+15,2.053317e+15,1.231507e+14,5.737468e+13,1.772130e+15,2.437543e+13,1.652797e+13,8.066030e+12,1.119782e+13
3,4.573182e+13,2.767290e+12,0.060511,0.638978,1.3568,265.4196,NaN,471.9549,NaN,1.3550,...,1.967870e+11,4.591844e+13,4.622727e+13,5.911390e+11,4.954510e+11,4.532730e+13,2.369800e+12,NaN,NaN,NaN
4,-4.739843e+13,-1.610339e+14,3.397452,0.306068,NaN,0.8690,NaN,NaN,NaN,NaN,...,-7.346602e+13,1.371765e+14,1.371765e+14,1.845749e+14,1.845749e+14,-4.739843e+13,-1.610339e+14,4.607902e+12,1.666914e+12,4.607902e+12


In [4]:
# convert all columns to float but Industry
cols_to_float = df.columns.drop('Industry')
df[cols_to_float] = df[cols_to_float].astype(float)
df.head()

,TSE_FY,RE_FY,y,Custom-Return,PtoB,PtoS,PtoFCFF,PtoE_ttm,PtoFCFE,PtoNAV,...,OP_FY,TA_MRQ,TA_FY,TL_MRQ,TL_FY,TSE_MRQ,RE_MRQ,FE_TTM,FE_MRQ,FE_FY
0,4.371690e+11,-1.348210e+14,308.395650,-0.046138,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.781578e+14,5.709565e+14,6.205788e+14,5.705193e+14,-4.242109e+13,-1.731840e+14,7.986821e+12,4.155223e+12,1.144012e+12
1,-1.125286e+15,-1.142159e+15,1.014995,NaN,NaN,NaN,48.5539,NaN,27.0587,NaN,...,NaN,3.158876e+15,3.158876e+15,4.284162e+15,4.284162e+15,-1.125286e+15,-1.142159e+15,8.043798e+13,3.522180e+13,8.043798e+13
2,1.995942e+15,2.536933e+14,0.127105,0.604888,1.1937,9.4587,NaN,10.2428,NaN,0.8344,...,2.219208e+14,1.895281e+15,2.053317e+15,1.231507e+14,5.737468e+13,1.772130e+15,2.437543e+13,1.652797e+13,8.066030e+12,1.119782e+13
3,4.573182e+13,2.767290e+12,0.060511,0.638978,1.3568,265.4196,NaN,471.9549,NaN,1.3550,...,1.967870e+11,4.591844e+13,4.622727e+13,5.911390e+11,4.954510e+11,4.532730e+13,2.369800e+12,NaN,NaN,NaN
4,-4.739843e+13,-1.610339e+14,3.397452,0.306068,NaN,0.8690,NaN,NaN,NaN,NaN,...,-7.346602e+13,1.371765e+14,1.371765e+14,1.845749e+14,1.845749e+14,-4.739843e+13,-1.610339e+14,4.607902e+12,1.666914e+12,4.607902e+12


In [5]:
# drop rows which has no value in TSE_FY or RE_FY column
df = df.dropna(subset=['TSE_FY', 'RE_FY'])
print(df.shape)

(820, 427)


In [6]:
print(df[df['y'] >= .5].shape)
print(df[df['y'] < .5].shape)
# drop y
df = df.drop(columns=['y'])

(390, 427)
(430, 427)


In [7]:
# create a y column RE_FY / TSE_FY > .5 = 1 else 0
df['y'] = np.where(df['RE_FY'] / df['TSE_FY'] > .5, 1, 0)
print(df[df['y'] >= .5].shape)
print(df[df['y'] < .5].shape)

(368, 427)
(452, 427)


/tmp/ipykernel_76271/2617393261.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['y'] = np.where(df['RE_FY'] / df['TSE_FY'] > .5, 1, 0)


In [8]:
# print nan proportion
print(df.isna().sum() / df.shape[0])

TSE_FY           0.000000
RE_FY            0.000000
Custom-Return    0.175610
PtoB             0.115854
PtoS             0.125610
                   ...   
RE_MRQ           0.000000
FE_TTM           0.286585
FE_MRQ           0.295122
FE_FY            0.273171
y                0.000000
Length: 427, dtype: float64


In [9]:
# df = df.fillna(df.mean())
# df = df.fillna(df.median())
# df = df.fillna(df.mode().iloc[0])

# fill NaN with 0
df = df.fillna(0)

print(df.isna().sum() / df.shape[0])

TSE_FY           0.0
RE_FY            0.0
Custom-Return    0.0
PtoB             0.0
PtoS             0.0
                ... 
RE_MRQ           0.0
FE_TTM           0.0
FE_MRQ           0.0
FE_FY            0.0
y                0.0
Length: 427, dtype: float64


In [10]:
# Scaler
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df.head()

,TSE_FY,RE_FY,Custom-Return,PtoB,PtoS,PtoFCFF,PtoE_ttm,PtoFCFE,PtoNAV,PtoD,...,TA_MRQ,TA_FY,TL_MRQ,TL_FY,TSE_MRQ,RE_MRQ,FE_TTM,FE_MRQ,FE_FY,y
0,0.360667,0.571374,4.482714e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.006307,0.000000,...,0.041453,0.040937,0.048209,0.044320,0.373735,0.549614,0.099292,0.117973,0.014222,0.0
1,0.000000,0.000000,4.953607e-07,0.000000,0.000000,0.000137,0.000000,0.000076,0.006307,0.015911,...,0.226490,0.226490,0.332808,0.332808,0.000000,0.000000,1.000000,1.000000,1.000000,1.0
2,1.000000,0.791744,1.112721e-06,0.000211,0.000018,0.000000,0.001280,0.000000,0.006455,0.000344,...,0.135891,0.147222,0.009567,0.004457,1.000000,0.661672,0.205475,0.229007,0.139211,0.0
3,0.375179,0.649416,1.147514e-06,0.000240,0.000513,0.000000,0.058967,0.000000,0.006548,0.012877,...,0.003291,0.003313,0.000046,0.000038,0.404020,0.649190,0.000000,0.000000,0.000000,0.0
4,0.345341,0.556506,8.077397e-07,0.000000,0.000002,0.000000,0.000000,0.000000,0.006307,0.000000,...,0.009835,0.009835,0.014338,0.014338,0.372017,0.556506,0.057285,0.047326,0.057285,1.0


In [11]:
# split data
x = df.drop(columns=['y'])
y = df['y']
x = np.array(x)
y = np.array(y)
# [samples, timesteps, features]
x = x.reshape(x.shape[0], 1, x.shape[1])

x_train = x[:int(x.shape[0] * .7)]
y_train = y[:int(y.shape[0] * .7)]
x_valid = x[int(x.shape[0] * .7):int(x.shape[0] * .85)]
y_valid = y[int(y.shape[0] * .7):int(y.shape[0] * .85)]
x_test = x[int(x.shape[0] * .85):]
y_test = y[int(y.shape[0] * .85):]

print(x_train.shape)
print(y_train.shape)
print("------")
print(x_valid.shape)
print(y_valid.shape)
print("------")
print(x_test.shape)
print(y_test.shape)

(574, 1, 426)
(574,)
------
(123, 1, 426)
(123,)
------
(123, 1, 426)
(123,)


In [12]:
# duplicate data
DUPLICATE = 8
x_train = np.repeat(x_train, DUPLICATE, axis=0)
y_train = np.repeat(y_train, DUPLICATE, axis=0)
print(x_train.shape)
print(y_train.shape)

(4592, 1, 426)
(4592,)


In [13]:
# LSTM
lstm = keras.Sequential()
# lstm.add(keras.layers.LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
lstm.add(keras.layers.LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2])))
lstm.add(keras.layers.Dense(1, activation='sigmoid'))
lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               284160    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 284289 (1.08 MB)
Trainable params: 284289 (1.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# train
history = lstm.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_valid, y_valid), shuffle=False)
# with no validation
# history = lstm.fit(x_train, y_train, epochs=100, batch_size=32, shuffle=False)

Epoch 1/100
72/72 [==============================] - 2s 7ms/step - loss: 0.6573 - accuracy: 0.6794 - val_loss: 0.6366 - val_accuracy: 0.6016
Epoch 2/100
72/72 [==============================] - 0s 3ms/step - loss: 0.6960 - accuracy: 0.5000 - val_loss: 0.6723 - val_accuracy: 0.5935
Epoch 3/100
72/72 [==============================] - 0s 3ms/step - loss: 0.6630 - accuracy: 0.6376 - val_loss: 0.6221 - val_accuracy: 0.8130
Epoch 4/100
72/72 [==============================] - 0s 3ms/step - loss: 0.6434 - accuracy: 0.6882 - val_loss: 0.5820 - val_accuracy: 0.6667
Epoch 5/100
72/72 [==============================] - 0s 3ms/step - loss: 0.6397 - accuracy: 0.6655 - val_loss: 0.5796 - val_accuracy: 0.6829
Epoch 6/100
72/72 [==============================] - 0s 3ms/step - loss: 0.6105 - accuracy: 0.7265 - val_loss: 0.5389 - val_accuracy: 0.7236
Epoch 7/100
72/72 [==============================] - 0s 3ms/step - loss: 0.5828 - accuracy: 0.7596 - val_loss: 0.5037 - val_accuracy: 0.7886
Epoch 8/100
7

In [15]:
# prin results
results = lstm.evaluate(x_test, y_test)
print(results)

4/4 [==============================] - 0s 2ms/step - loss: 1.6016 - accuracy: 0.6585
[1.6015721559524536, 0.6585366129875183]


In [19]:
# plot loss
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='loss'))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='val_loss'))
fig.update_layout(title='Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

In [20]:
# plot accuracy
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines', name='accuracy'))
fig.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines', name='val_accuracy'))
fig.update_layout(title='Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy')
fig.show()